In [1]:
library("btergm")
library("dplyr")
library("Matrix")
library("statnet")  
library("texreg") 
library("data.table")
library("metafor")

Loading required package: xergm.common
Loading required package: ergm
Loading required package: network
network: Classes for Relational Data
Version 1.16.0 created on 2019-11-30.
copyright (c) 2005, Carter T. Butts, University of California-Irvine
                    Mark S. Handcock, University of California -- Los Angeles
                    David R. Hunter, Penn State University
                    Martina Morris, University of Washington
                    Skye Bender-deMoll, University of Washington
 For citation information, type citation("network").
 Type help("network-package") to get started.


ergm: version 3.10.4, created on 2019-06-10
Copyright (c) 2019, Mark S. Handcock, University of California -- Los Angeles
                    David R. Hunter, Penn State University
                    Carter T. Butts, University of California -- Irvine
                    Steven M. Goodreau, University of Washington
                    Pavel N. Krivitsky, University of Wollongong
     

In [5]:
#Data Preparation 

#Generate edge matrix and adjust for missing players
create_round_edges <- function(data, players){
  players_in_game <- union(unique(data$player_id), unique(data$other_id))
  edge_mat <- matrix(0,nrow=players,ncol=players)
  data <- adjust_missing_players(data, players_in_game)
  for(j in 1:nrow(data)){
    player_id <- data[[j, "player_id"]]
    other_id <- data[[j, "other_id"]]
    edge_mat[player_id, other_id] <- 1
  }
  return(edge_mat)
}

#Add Endowment as a vertex attribute to the network
add_endowment <- function(network, data){
  endowment <- unique(data[(data$round==1), ])
  endowment <- endowment[ , c("player_id", "endowment")][order(endowment$player_id), ]
  network <- set.vertex.attribute(network, "endowment", endowment[,2])
  return(network)
}

#Adjust wealth for missing players and add it to the network
add_wealth <- function(network, data, round){
  wealth_data <- as.data.table(data)
  wealth_data <- wealth_data[ , wealth := shift(final_wealth, fill=0), by=c('treatment', 'group_id', 'player_id')]
  wealth_data <- transform(wealth_data, wealth=sqrt(wealth))
  players_in_game <- wealth_data[(wealth_data$round==1), ]$player_id
  wealth_data <- adjust_missing_players(wealth_data, players_in_game)
  setDF(wealth_data)
  round_wealth <- get_round_wealth(wealth_data, round)
  network <- set.vertex.attribute(network, "wealth", round_wealth)
  return(network)
}

#Adjust ID's to account for missing players
adjust_missing_players <- function(data, players_in_game){
  num_players <- length(players_in_game)
  if (num_players < max(players_in_game)) {
    x <- setdiff(seq(1, max(players_in_game)), players_in_game) 
    data$player_id[data$player_id > x] <- data$player_id[data$player_id > x] - 1
    if("other_id" %in% colnames(data))
    {
      data$other_id[data$other_id > x] <- data$other_id[data$other_id > x] - 1
    }
  }
  return(data)
}

#Returns the wealth of a player for in a specific round
get_round_wealth <- function(data, current_round){
  round_data <- data[(data$round==current_round), ]
  return(round_data[,"wealth"])
}

#Generate the network in the correct format and add the outdegree as a vertex atrribute
get_btergm_data <- function(data1, data2){
  network <- list()
  players_in_game <- union(unique(data1$player_id), unique(data1$other_id))
  num_players <- length(players_in_game)
  for(i in 1:20){
    round_edges <- create_round_edges(data1[(data1$round == i), ], num_players)
    network[[i]] <- network(round_edges)
    network[[i]] <- add_endowment(network[[i]], data2)
    network[[i]] <- add_wealth(network[[i]], data2, i)
    outdegree <- degree(network[[i]], cmode = "outdegree")
    network[[i]] <-  (network[[i]],
                                         "outdegree", outdegree)
  }
  return(network)
}

#   Metafor package

#Extract model data for metafor
get_metafor_data <- function(group_models){
  terms <- btergm.se(group_models[[1]])
  yis <-matrix(list(), nrow=nrow(terms), ncol=length(group_models))
  seis <- matrix(list(), nrow=nrow(terms), ncol=length(group_models))
  for(i in 1:length(group_models)){
    model <- group_models[[i]]
    stats <- btergm.se(model)
    for(j in 1:nrow(stats)){
      yis[i,j] = stats[j,1] #Gets the coef estimate
      seis[i,j] = stats[j,2] #Gets the coef standart error
    }
  }
  return(export_coefs(btergm08(yis,seis,terms),terms))
}


#Run meta-analysis using all group models for each coefficient
btergm08 <- function(yis,seis,terms){
  yis <- split(yis, rep(1:ncol(yis), each = nrow(yis)))
  seis <- split(seis, rep(1:ncol(seis), each = nrow(seis)))
  coefs <- list()
  for(i in 1:nrow(terms)){
    yi <- yis[i]
    sei <- seis[i]
    yi<- unlist(yi, use.names=FALSE)
    sei<- unlist(sei, use.names=FALSE)
    coef <- rownames(terms)[i]
    new.term <- data.frame(yi, sei)
    meta<- rma.uni(yi, sei, data = new.term, method = "REML", level = 95)
    coefs[[coef]] <- meta
  }
  return (coefs)
}

#Create coefficient matrix
export_coefs <- function(coefs,terms){
  mat <- matrix(list(), nrow = nrow(terms), ncol = 5)
  rownames(mat) <- rownames(terms)
  for(name in rownames(terms)){
    term <- coefs[[name]]
    est <- as.numeric(term["beta"])
    se <- as.numeric(term["se"])
    lb <-as.numeric(term["ci.lb"])
    ub <-as.numeric(term["ci.ub"])
    lev <- 1-as.numeric(term["level"])
    mat[name,] <- c(est,se,lb,ub,lev)
  }
  colnames(mat) <- c("estimate","se","mu-min","mu-plus","alpha-mu")
  return(mat)
}





Load Data #TODO

In [6]:
interact_data <- read.csv("data/interactions.csv", header=T)
wealth_data <- read.csv("data/finwealth_from_interactions.csv", header=T)

Run Models #TODO

In [7]:
all_networks <- list()
all_models <- list()
for (t in unique(interact_data$treatment)) {
  group_networks <- list()
  group_models <- list()
  test <- list()
  for (g in unique(interact_data[(interact_data$treatment==t), ]$group_id)) {
    filtered_data1 <- interact_data[(interact_data$treatment==t) & (interact_data$group_id==g), ]
    filtered_data2 <- wealth_data[(wealth_data$treatment==t) & (wealth_data$group_id==g), ]
    btergm_network <- get_btergm_data(filtered_data1, filtered_data2)
    btergm_model <- btergm(btergm_network ~ edges + mutual + ttriple +
                          + nodecov("outdegree") +
                          nodeicov("endowment") + nodeocov("endowment") + nodematch("endowment")
                        + nodeicov("wealth") + nodeocov("wealth") + nodematch("wealth"), R = 100)
    group_networks[[paste(t, '_', g, sep="")]] <- btergm_network
    group_models[[paste(t, '_', g, sep="")]] <- btergm_model

  }
  all_models[[paste(t, sep="")]] <- group_models
  #Export metafor coefficients
  write.csv(get_metafor_data(group_models), paste('btergm_version2_params_', t, '.csv', sep=''))
  all_networks[[paste(t, sep="")]] <- group_networks
}


No covariates provided.

All networks are conformable.

Starting pseudolikelihood estimation with 100 bootstrapping replications on a single computing core...
Warning message:
“`set_attrs()` is deprecated as of rlang 0.3.0
This warning is displayed once per session.”Done.

No covariates provided.

All networks are conformable.

Starting pseudolikelihood estimation with 100 bootstrapping replications on a single computing core...
Done.

No covariates provided.

All networks are conformable.

Starting pseudolikelihood estimation with 100 bootstrapping replications on a single computing core...
Done.
Warning message in data$player_id > x:
“longer object length is not a multiple of shorter object length”Warning message in data$player_id > x:
“longer object length is not a multiple of shorter object length”Warning message in data$other_id > x:
“longer object length is not a multiple of shorter object length”Warning message in data$other_id > x:
“longer object length is not a multiple of sh


Starting pseudolikelihood estimation with 100 bootstrapping replications on a single computing core...
Done.

No covariates provided.

All networks are conformable.

Starting pseudolikelihood estimation with 100 bootstrapping replications on a single computing core...
Done.

No covariates provided.

All networks are conformable.

Starting pseudolikelihood estimation with 100 bootstrapping replications on a single computing core...
Done.

No covariates provided.

All networks are conformable.

Starting pseudolikelihood estimation with 100 bootstrapping replications on a single computing core...
Done.
Warning message in btergm.se(group_models[[1]]):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading b

“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”

In [ ]:
#Run model for combined groups
for (t in unique(interact_data$treatment)){
  combined_group_network <- list()
  group <- all_networks[[t]]
  for(network in group){
    combined_group_network <- append(combined_group_network, network)
  }
  btergm_model <- btergm(combined_group_network ~ edges + mutual + ttriple +
                          + nodecov("outdegree") + nodeicov("endowment") 
                          + nodeocov("endowment") + nodematch("endowment")
                          + nodeicov("wealth") + nodeocov("wealth") + nodematch("wealth"), R = 100)
  
  write.csv(cbind(btergm.se(btergm_model), confint(btergm_model,level=0.9)[, 2:3]), paste('btergm_version1_params_', t, '.csv', sep=''))
}

